In [ ]:
# YOLOv8 Training Script

from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n variant (custom lightweight model)
model = YOLO('../src/human_detector/models/yolov11n.pt')


# Training with simple parameters
#    - data: path to your data.yaml
#    - epochs, imgsz, project, name: same CLI args
# results = model.train(
#     data    = 'data.yaml',
#     epochs  = 1,
#     imgsz   = 640,
#     project = 'human_robot',
#     name    = 'exp1'
# )

# Low-light training hyperparameters as a Python dictionary
custom_hyp = {
    'epochs': 120,
    'patience': 20,
    'imgsz': 416,
    'batch': 2,
    'device': 0,
    'half': True,
    'cache': False,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,     # Brightness variation for low-light
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
    'copy_paste': 0.0
}

# Train the model with custom augmentation settings and longer epochs
results = model.train(
    data='data.yaml',
    overrides=custom_hyp,
    project='human_robot',
    name='exp_lowlight'
)

# Save trained model path
save_dir = str(results.save_dir)
best_pt = os.path.join(save_dir, 'weights', 'best.pt')
print(f"✅  Trained checkpoint saved to: {best_pt}")


SyntaxError: '[31m[1mhyp[0m' is not a valid YOLO argument. 

    Arguments received: ['yolo', '--f=/run/user/1000/jupyter/runtime/kernel-v3426a4afb468cfa9a7934d7780764969ed7794264.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'pose', 'detect', 'segment', 'classify', 'obb'})
                MODE (required) is one of frozenset({'track', 'train', 'predict', 'benchmark', 'val', 'export'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=yolo11n-seg.pt source='https:

# 🧠 YOLOv8 Training Metrics Cheatsheet

## 🔢 Loss Curves

| Metric            | Meaning                                                | Goal / Good Trend                |
|-------------------|--------------------------------------------------------|----------------------------------|
| `train/box_loss`  | Bounding box regression loss (training)                | ↓ Steady decline                 |
| `train/cls_loss`  | Classification loss (training)                         | ↓ Should fall and stabilize     |
| `train/dfl_loss`  | Distribution Focal Loss (for bounding box quality)     | ↓ Smooth drop                   |
| `val/box_loss`    | Box loss on validation set                             | ↓ Best indicator of generalization |
| `val/cls_loss`    | Classification loss on validation                      | ↓ Falling → Good model fit      |
| `val/dfl_loss`    | DFL loss on validation                                 | ↓ Matches train trend = no overfit |

---

## 📈 Detection Metrics

| Metric                | Description                                                 | Goal / Interpretation           |
|------------------------|-------------------------------------------------------------|----------------------------------|
| `metrics/precision(B)` | % of correct detections out of all predictions              | ↑ High = few false positives     |
| `metrics/recall(B)`    | % of actual objects detected                                | ↑ High = few missed detections   |
| `metrics/mAP50(B)`     | Accuracy of boxes at 50% IoU threshold                     | ↑ Main object detection metric   |
| `metrics/mAP50-95(B)`  | Average mAP over IoU 0.5 to 0.95 (stricter)                | ↑ Robust performance measure     |

---

## 🧪 Ideal Curve Patterns

- **Losses**: should steadily decrease.
- **Validation loss** rising while training loss decreases → possible **overfitting**.
- **Precision & Recall**: should increase and plateau near 1.0.
- **mAP**: aim for **>0.90 mAP50** and **>0.80 mAP50-95** for high-quality results.

---

## ⚠️ Tips

- 🧊 Use `half=True` for mixed precision training on supported GPUs.
- 🧠 Watch `val/box_loss` and `mAP50-95` to catch overfitting early.
- 🧪 If metrics plateau early, try increasing `epochs`, `imgsz`, or improving labels.
- 💾 Check `runs/train/.../results.png` for curve visualization.

---
